In [1]:
# 必要に応じて
# !pip install --upgrade pip
# !pip install numpy scikit-learn tensorflow matplotlib pillow pandas

In [2]:
# 機械学習のライブラリ関連をインポート
import pandas as pd
import numpy as np

# ランダムにシャッフルして，学習・テストに分割するモジュール
from sklearn.model_selection import ShuffleSplit

# 深層学習のライブラリをインポート
import tensorflow as tf
import tensorflow.keras as keras

#表示系のインポートと設定
%matplotlib inline
import matplotlib.pyplot as plt

# ファイル操作
import os

# 画像操作
from PIL import Image

# 時間取得
import datetime

In [47]:
# DIRS
DATASET_NUM = 1
DIRS_DATASET = "../Training/Datasets" + str(DATASET_NUM) + "/"

In [48]:
file_names = []
    
# フォルダ内のファイルを取得
files = os.listdir(DIRS_DATASET)

# ファイル名を配列に格納
for file in files:
    file_names.append(file)

# Pythonリスト型をnumpy.ndarray型に変換
file_names = np.array(file_names)


In [49]:
file_names

array(['4525491507698.jpeg'], dtype='<U18')

### 画像をnumpy配列にするための関数

In [52]:
def jpeg_to_numpy(image_path):
    # JPEG画像を開く
    image = Image.open(image_path)
    # NumPy配列に変換
    numpy_array = np.array(image)
    
    return numpy_array

def convert_to_grayscale(numpy_array):
    # グレーと言わず2値化
    grayscale_array = np.where(numpy_array <= 128, 0, 255)
    # plt.imshow(grayscale_array) # こいつらのせいで処理が重かった。出力系は要注意
    # print(grayscale_array)
    return grayscale_array


### 画像を配列にしてよしなに

In [126]:
X, y = [], []

In [127]:
for file_name in file_names:
    numpy_array = jpeg_to_numpy(DIRS_DATASET + file_name)
    grayscale_array = convert_to_grayscale(numpy_array)
    for i in range(13):
        grayscale_array1 = np.where(np.all(grayscale_array == [0, 0, 0], axis=-1), 0, 255)
        grayscale_array2 = np.where(grayscale_array1 == 255, i+1, 0)
        X.append(grayscale_array2)
        y.append(file_name[i])

In [128]:
X = np.array(X)
X = X.squeeze()
y = np.array(y, dtype=int)

In [129]:
X

array([[ 0,  1,  0, ...,  1,  1,  0],
       [ 0,  2,  0, ...,  2,  2,  0],
       [ 0,  3,  0, ...,  3,  3,  0],
       ...,
       [ 0, 11,  0, ..., 11, 11,  0],
       [ 0, 12,  0, ..., 12, 12,  0],
       [ 0, 13,  0, ..., 13, 13,  0]])

In [130]:
y

array([4, 5, 2, 5, 4, 9, 1, 5, 0, 7, 6, 9, 8])

In [131]:
# ラベルデータをone-hotベクトルに直す
labels = {
    0: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
    1: [0, 1, 0, 0, 0, 0, 0, 0, 0, 0], 
    2: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0], 
    3: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0], 
    4: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0], 
    5: [0, 0, 0, 0, 0, 1, 0, 0, 0, 0], 
    6: [0, 0, 0, 0, 0, 0, 1, 0, 0, 0], 
    7: [0, 0, 0, 0, 0, 0, 0, 1, 0, 0], 
    8: [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], 
    9: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1], 
}

y = np.array(list(map(lambda v : labels[v] , y)))

In [132]:
y

array([[0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]])

### データの分割

In [133]:
ss = ShuffleSplit(n_splits=1,      # 分割を1個生成
                  train_size=0.8,  # 学習
                  test_size =0.2,  # テスト
                  random_state=0)  # 乱数種（再現用）

# 学習データとテストデータのインデックスを作成
train_index, test_index = next(ss.split(X))

X_train, X_test = X[train_index], X[test_index] # 学習データ，テストデータ
y_train, y_test = y[train_index], y[test_index] # 学習データのラベル，テストデータのラベル

In [134]:
X

array([[ 0,  1,  0, ...,  1,  1,  0],
       [ 0,  2,  0, ...,  2,  2,  0],
       [ 0,  3,  0, ...,  3,  3,  0],
       ...,
       [ 0, 11,  0, ..., 11, 11,  0],
       [ 0, 12,  0, ..., 12, 12,  0],
       [ 0, 13,  0, ..., 13, 13,  0]])

In [135]:
y

array([[0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]])

In [137]:
X_train.shape[0], X_train.shape[1]

(10, 112)

In [138]:
X_train

array([[ 0, 11,  0, ..., 11, 11,  0],
       [ 0,  3,  0, ...,  3,  3,  0],
       [ 0,  9,  0, ...,  9,  9,  0],
       ...,
       [ 0,  1,  0, ...,  1,  1,  0],
       [ 0,  6,  0, ...,  6,  6,  0],
       [ 0, 13,  0, ..., 13, 13,  0]])

### ニューラルネットワークの構築

In [142]:

# 学習し、テストデータで評価し、スコアを表示する
# 引数は、中間層の数、バッチサイズ、epoch数

def fit_epoch(neuron, batch, epochs, ckpt_period):
    ver_name = "231229-v3"
    
    # チェックポイントの設定
    dt_now = datetime.datetime.now()
    checkpoint_path = "./training_ckpt_v" + ver_name + "_d" + str(DATASET_NUM) + "_n" + str(neuron)  + "_b" + str(batch) + "_e" + str(epochs) + "_" + dt_now.strftime('%Y%m%d%H%M%S') + "/cp-{epoch:06d}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    cp_callback = tf.keras.callbacks.ModelCheckpoint(
        checkpoint_path,
        verbose=1,
        save_weights_only=True,
        period=ckpt_period  # 重みをckpt_periodエポックごとに保存します
    )


    # レイヤーのオブジェクトを作成
    Dense = keras.layers.Dense

    # モデルの構造を定義
    model = keras.models.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(112, )))
    model.add(Dense(neuron, activation='relu'))
    
    # 畳み込み層を追加
    # model.add(tf.keras.layers.Flatten(tf.keras.layers.Conv2D(filters=neuron, kernel_size=(3, 3), activation='relu', input_shape=(1, 112, 3))))

    model.add(Dense(10, activation='softmax')) # 10つのラベルがありsoftmaxで最後の層作る

    # モデルを構築
    model.compile(
        loss='categorical_crossentropy',
        optimizer='Adamax',
        metrics=['accuracy']
    )

    # 必要に応じてチェックポイントから再開
    # model.load_weights("")

    # 学習を実行
    hist = model.fit(X_train, y_train,
        batch_size=batch, # 誤差逆伝播法をするときの1回当たりのデータ数
        epochs=epochs,
        callbacks=[cp_callback],
        verbose=1,
        validation_split=0.1)
    
   # モデルを評価
    score = model.evaluate(X_test, y_test, verbose=1)
    print('正解率=', score[1], 'loss=', score[0])
    
     # 学習の様子をグラフへ描画 
    # 正解率の推移をプロット
    plt.plot(hist.history['accuracy'])
    plt.plot(hist.history['val_accuracy'])
    plt.title('Accuracy')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

    # ロスの推移をプロット
    plt.plot(hist.history['loss'])
    plt.plot(hist.history['val_loss'])
    plt.title('Loss')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()   

In [143]:
print(DATASET_NUM)

1


In [144]:
%%time
fit_epoch(16, 512, 400000, 10000)

Epoch 1/400000
1/1 [==============================] - 1s 966ms/step - loss: 7.9414 - accuracy: 0.0000e+00 - val_loss: 16.3638 - val_accuracy: 0.0000e+00
Epoch 2/400000
1/1 [==============================] - 0s 49ms/step - loss: 7.2185 - accuracy: 0.0000e+00 - val_loss: 15.2457 - val_accuracy: 0.0000e+00
Epoch 3/400000
1/1 [==============================] - 0s 42ms/step - loss: 6.5229 - accuracy: 0.0000e+00 - val_loss: 14.1533 - val_accuracy: 0.0000e+00
Epoch 4/400000
1/1 [==============================] - 0s 51ms/step - loss: 5.8646 - accuracy: 0.0000e+00 - val_loss: 13.1151 - val_accuracy: 0.0000e+00
Epoch 5/400000
1/1 [==============================] - 0s 49ms/step - loss: 5.2624 - accuracy: 0.0000e+00 - val_loss: 12.2183 - val_accuracy: 0.0000e+00
Epoch 6/400000
1/1 [==============================] - 0s 43ms/step - loss: 4.7514 - accuracy: 0.0000e+00 - val_loss: 11.6591 - val_accuracy: 0.0000e+00
Epoch 7/400000
1/1 [==============================] - 0s 49ms/step - loss: 4.3734 - acc

KeyboardInterrupt: 